In [ ]:
import os
base_dir = '/content/drive/MyDrive/chest_xray'  # Update as needed

# Define subdirectories for training, validation and testing
train_dir = os.path.join(base_dir, 'train')
val_dir   = os.path.join(base_dir, 'val')
test_dir  = os.path.join(base_dir, 'test')

In [ ]:
# Check for GPU availability in Colab
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    print('GPU device not found. Please ensure that you have enabled GPU runtime in Colab.')
else:
    print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
from tensorflow.keras import layers, models

def build_transfer_model(base_model, input_shape, num_classes=1):
    """
    Freeze the base model and add custom classification layers.
    Assumes binary classification (num_classes=1 with sigmoid output).
    """
    base_model.trainable = False  # Freeze the pre-trained base model

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='sigmoid')  # Sigmoid for binary classification
    ])

    # Compile the model
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model


In [ ]:
from tensorflow.keras.applications import VGG19

# Data generators for VGG19 (input size 224x224)
img_height, img_width = 224, 224
batch_size = 32

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)




Found 216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 132 images belonging to 2 classes.


In [ ]:
# Load the VGG19 base model
vgg19_base = VGG19(include_top=False, weights='imagenet', input_shape=(img_height, img_width, 3))
vgg19_model = build_transfer_model(vgg19_base, (img_height, img_width, 3))



80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [ ]:
# Display the model architecture
vgg19_model.summary()



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg19 (Functional)                   │ (None, 7, 7, 512)           │      20,024,384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          65,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 20,090,177 (76.64 MB)

 Trainable params: 65,793 (257.00 KB)

 Non-trainable params: 20,024,384 (76.39 MB)

In [ ]:
# Train the VGG19 model
epochs = 10
history_vgg19 = vgg19_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    epochs=epochs
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 74s 10s/step - accuracy: 0.4927 - loss: 0.8393 - val_accuracy: 0.5000 - val_loss: 0.6881
Epoch 2/10
1/6 ━━━━━━━━━━━━━━━━━━━━ 1s 242ms/step - accuracy: 0.4375 - loss: 0.8091

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.4375 - loss: 0.8091 - val_accuracy: 0.5000 - val_loss: 0.6918
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 14s 797ms/step - accuracy: 0.5341 - loss: 0.7286 - val_accuracy: 0.5000 - val_loss: 0.6659
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.7500 - loss: 0.5403 - val_accuracy: 0.5000 - val_loss: 0.6657
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.5914 - loss: 0.6838 - val_accuracy: 0.7500 - val_loss: 0.6169
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.6875 - loss: 0.5784 - val_accuracy: 0.6875 - val_loss: 0.6155
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 740ms/step - accuracy: 0.6329 - loss: 0.6038 - val_accuracy: 0.6875 - val_loss: 0.6075
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.7500 - loss: 0.5417 - val_accuracy: 0.6875 - val_loss: 0.6071
Epoch 9/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 755ms/step - accuracy: 0.6909 - loss: 0.5396 - val_accuracy: 0.7500 - val_loss: 0.5789
Epoch 1

In [ ]:
from tensorflow.keras.applications import DenseNet201

# The data generators remain the same as for VGG19 since the input size is 224x224.
# (train_generator, val_generator, test_generator are already defined)

# Load the DenseNet201 base model
densenet_base = DenseNet201(include_top=False, weights='imagenet', input_shape=(img_height, img_width, 3))
densenet_model = build_transfer_model(densenet_base, (img_height, img_width, 3))



74836368/74836368 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [ ]:
# Display the model architecture
densenet_model.summary()

# Train the DenseNet201 model
history_densenet = densenet_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    epochs=epochs
)



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ densenet201 (Functional)             │ (None, 7, 7, 1920)          │      18,321,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 1920)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │         245,888 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 18,568,001 (70.83 MB)

 Trainable params: 246,017 (961.00 KB)

 Non-trainable params: 18,321,984 (69.89 MB)

Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 102s 11s/step - accuracy: 0.6160 - loss: 0.6338 - val_accuracy: 0.7500 - val_loss: 0.4247
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.8438 - loss: 0.3844 - val_accuracy: 0.7500 - val_loss: 0.4460
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 717ms/step - accuracy: 0.8697 - loss: 0.3766 - val_accuracy: 0.9375 - val_loss: 0.3659
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.7500 - loss: 0.4299 - val_accuracy: 0.8125 - val_loss: 0.3601
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 768ms/step - accuracy: 0.8700 - loss: 0.3585 - val_accuracy: 0.8750 - val_loss: 0.3577
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.9062 - loss: 0.1821 - val_accuracy: 0.8750 - val_loss: 0.3653
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 834ms/step - accuracy: 0.9008 - loss: 0.2259 - val_accuracy: 0.9375 - val_loss: 0.3418
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 133ms/step - accuracy: 0.8750 - loss: 0.2431 - val_accuracy: 0.8750 - val_loss: 0

In [ ]:
# Evaluate the DenseNet201 model on the test set
test_loss_densenet, test_acc_densenet = densenet_model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print('DenseNet201 Test accuracy:', test_acc_densenet)

4/4 ━━━━━━━━━━━━━━━━━━━━ 60s 15s/step - accuracy: 0.7729 - loss: 0.4746
DenseNet201 Test accuracy: 0.8203125


In [ ]:
# prompt: save model as h5

# Save the trained DenseNet201 model
densenet_model.save('densenet_model.h5')


In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import preprocess_input



In [ ]:

# Function to load and preprocess the image
def load_and_preprocess_img(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img = image.img_to_array(img)
    # Expand dims and preprocess (VGG expects BGR ordering and mean subtraction)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img



In [ ]:
# Function to generate Grad-CAM heatmap
def get_gradcam_heatmap(model, img_array, last_conv_layer_name, pred_index=None):
    # Create a model that maps the input image to the activations of the last conv layer
    # as well as the output predictions
    grad_model = Model(
        inputs=[model.inputs],
        outputs=[model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Compute the gradient of the top predicted class for the input image
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    # Compute the gradient of the class output with respect to the feature map
    grads = tape.gradient(class_channel, conv_outputs)

    # Compute the channel-wise mean of the gradients
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Multiply each channel in the feature map array by "how important this channel is" with respect to the class
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization, normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()



In [ ]:
# Function to overlay the heatmap on original image
def overlay_heatmap(img_path, heatmap, alpha=0.4, colormap=cv2.COLORMAP_JET):
    # Load original image
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Resize heatmap to match the image size
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    # Convert heatmap to RGB
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, colormap)
    # Overlay heatmap on original image
    superimposed_img = cv2.addWeighted(img, 1 - alpha, heatmap, alpha, 0)
    return superimposed_img



In [ ]:
# Example usage:
# 1. Load your trained VGG model (if not already in memory)
# For demonstration, we are using a pre-trained VGG19 model without top classifier.
# (Replace with your own model if needed.)
from tensorflow.keras.applications import VGG19
model = VGG19(weights='imagenet')
last_conv_layer_name = 'block5_conv4'  # Last convolutional layer in VGG19

# 2. Provide the path to the input image
img_path = '/content/drive/MyDrive/chest_xray/train/PNEUMONIA/person10_bacteria_43.jpeg'  # Update with your image path

# 3. Load and preprocess the image
img_array = load_and_preprocess_img(img_path, target_size=(224, 224))

# 4. Generate Grad-CAM heatmap
heatmap = get_gradcam_heatmap(vgg19_model, img_array, last_conv_layer_name)

# 5. Overlay the heatmap on the original image
superimposed_img = overlay_heatmap(img_path, heatmap, alpha=0.4)

# 6. Display the result
plt.figure(figsize=(10, 10))
plt.imshow(superimposed_img)
plt.axis('off')
plt.title('Grad-CAM')
plt.show()